In [2]:
# import via notebook
!mongoimport --type json -d wine -c ratings --drop --jsonArray ../Resources/clean_wine_data_final.json

2024-05-09T16:32:13.879-0700	Failed: open ..\Resources\clean_wine_data_final.json: The system cannot find the file specified.
2024-05-09T16:32:13.880-0700	0 document(s) imported successfully. 0 document(s) failed to import.


In [92]:
# Importing Necessary Libraries
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [7]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [8]:
db = mongo['wine']
db.list_collection_names()

['ratings']

In [9]:
#assign the collection to a variable
wine_df = db['ratings']

In [10]:
cursor = db["ratings"].find({})
json_data = list(cursor)

In [11]:
wine_df = pd.DataFrame(json_data)
wine_df.head()

,_id,country,description,points,price,province,region,title,variety,winery,rating_category,type,vintage
0,663af692e8d6239274b71533,France,This has great depth of flavor with its fresh ...,87,27,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,Good,White,2012
1,663af692e8d6239274b71534,US,"Rustic and dry, this has flavors of berries, c...",86,50,California,Dry Creek Valley,Envolve 2010 Puma Springs Vineyard Red (Dry Cr...,Red Blend,Envolve,Good,Red,2010
2,663af692e8d6239274b71535,US,"Soft, supple plum envelopes an oaky structure ...",87,19,California,Napa Valley,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,Good,Red,2011
3,663af692e8d6239274b71536,US,"This shows a tart, green gooseberry flavor tha...",86,20,California,Sonoma Valley,Envolve 2011 Sauvignon Blanc (Sonoma Valley),Sauvignon Blanc,Envolve,Good,White,2011
4,663af692e8d6239274b71537,Italy,This concentrated Cabernet offers aromas of cu...,86,21,Sicily & Sardinia,Sicilia,Feudi del Pisciotto 2010 Missoni Cabernet Sauv...,Cabernet Sauvignon,Feudi del Pisciotto,Good,Red,2010


In [12]:
# Drop the non-beneficial ID columns
wine_df = wine_df.drop(columns = ['_id','description','title', 'winery', 'region'])
wine_df.head()

,country,points,price,province,variety,rating_category,type,vintage
0,France,87,27,Alsace,Pinot Gris,Good,White,2012
1,US,86,50,California,Red Blend,Good,Red,2010
2,US,87,19,California,Cabernet Sauvignon,Good,Red,2011
3,US,86,20,California,Sauvignon Blanc,Good,White,2011
4,Italy,86,21,Sicily & Sardinia,Cabernet Sauvignon,Good,Red,2010


In [13]:
wine_df['target'] =wine_df['points']>=90
wine_df.tail()

,country,points,price,province,variety,rating_category,type,vintage,target
77926,US,90,20,California,Chardonnay,Very Good,White,2013,True
77927,France,90,28,Alsace,Pinot Gris,Very Good,White,2013,True
77928,France,90,57,Alsace,Pinot Gris,Very Good,White,2010,True
77929,US,90,35,California,Chardonnay,Very Good,White,2013,True
77930,US,90,75,Oregon,Pinot Noir,Very Good,Red,2004,True


In [14]:
wine_df = wine_df.drop(columns = ['points','rating_category', 'province', 'type'])

In [35]:
wine_df.head()

,country,price,variety,vintage,target
0,France,27,Pinot Gris,2012,False
1,US,50,Red Blend,2010,False
2,US,19,Cabernet Sauvignon,2011,False
3,US,20,Sauvignon Blanc,2011,False
4,Italy,21,Cabernet Sauvignon,2010,False


In [36]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77931 entries, 0 to 77930
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  77931 non-null  object
 1   price    77931 non-null  int64 
 2   variety  77931 non-null  object
 3   vintage  77931 non-null  object
 4   target   77931 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 2.5+ MB


In [15]:
wine_df['vintage'] = wine_df['vintage'].replace('No Year', pd.NA)


In [38]:
wine_df.dropna()

,country,price,variety,vintage,target
0,France,27,Pinot Gris,2012,False
1,US,50,Red Blend,2010,False
2,US,19,Cabernet Sauvignon,2011,False
3,US,20,Sauvignon Blanc,2011,False
4,Italy,21,Cabernet Sauvignon,2010,False
...,...,...,...,...,...
77926,US,20,Chardonnay,2013,True
77927,France,28,Pinot Gris,2013,True
77928,France,57,Pinot Gris,2010,True
77929,US,35,Chardonnay,2013,True


In [16]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77931 entries, 0 to 77930
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  77931 non-null  object
 1   price    77931 non-null  int64 
 2   variety  77931 non-null  object
 3   vintage  75256 non-null  object
 4   target   77931 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 2.5+ MB


In [19]:
wine_df.loc[wine_df['vintage'].isna(),]

,country,price,variety,vintage,target
19,France,58,Champagne Blend,NaN,False
39,France,55,Champagne Blend,NaN,False
179,France,42,Champagne Blend,NaN,False
180,Spain,11,Sparkling Blend,NaN,False
214,France,30,Champagne Blend,NaN,False
...,...,...,...,...,...
77780,US,22,Sparkling Blend,NaN,False
77790,US,24,Sparkling Blend,NaN,False
77798,France,49,Champagne Blend,NaN,True
77811,France,70,Champagne Blend,NaN,True


In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
wine_dummies_df = pd.get_dummies(wine_df, dtype = int)
wine_dummies_df.head()

,price,target,country_Argentina,country_Australia,country_Canada,country_France,country_Italy,country_Spain,country_US,variety_Bordeaux-style Red Blend,...,vintage_2008,vintage_2009,vintage_2010,vintage_2011,vintage_2012,vintage_2013,vintage_2014,vintage_2015,vintage_2016,vintage_2017
0,27,False,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,50,False,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,19,False,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,20,False,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,21,False,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [18]:
wine_dummies_df.columns

Index(['price', 'target', 'country_Argentina', 'country_Australia',
       'country_Canada', 'country_France', 'country_Italy', 'country_Spain',
       'country_US', 'variety_Bordeaux-style Red Blend',
       'variety_Cabernet Franc', 'variety_Cabernet Sauvignon',
       'variety_Champagne Blend', 'variety_Chardonnay', 'variety_Malbec',
       'variety_Merlot', 'variety_Nebbiolo', 'variety_Pinot Gris',
       'variety_Pinot Noir', 'variety_Red Blend',
       'variety_Rhône-style Red Blend', 'variety_Riesling', 'variety_Rosé',
       'variety_Sangiovese', 'variety_Sauvignon Blanc',
       'variety_Sparkling Blend', 'variety_Syrah', 'variety_Tempranillo',
       'variety_White Blend', 'variety_Zinfandel', 'vintage_1872',
       'vintage_1882', 'vintage_1887', 'vintage_1904', 'vintage_1919',
       'vintage_1945', 'vintage_1985', 'vintage_1986', 'vintage_1987',
       'vintage_1988', 'vintage_1989', 'vintage_1990', 'vintage_1991',
       'vintage_1992', 'vintage_1993', 'vintage_1994', 'vi

In [28]:
# Split our preprocessed data into our features and target arrays
y = wine_dummies_df['target']
X = wine_dummies_df.drop (columns = 'target')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
X_test

,price,country_Argentina,country_Australia,country_Canada,country_France,country_Italy,country_Spain,country_US,variety_Bordeaux-style Red Blend,variety_Cabernet Franc,...,vintage_2008,vintage_2009,vintage_2010,vintage_2011,vintage_2012,vintage_2013,vintage_2014,vintage_2015,vintage_2016,vintage_2017
27137,95,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
66081,20,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
47734,40,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54891,60,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
67218,45,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,60,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39751,29,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
35320,30,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
70090,50,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [31]:
wine_df.iloc[27137,:]

country                    US
price                      95
variety    Cabernet Sauvignon
vintage                  2013
target                  False
Name: 27137, dtype: object

In [32]:
test_df = X_test.reset_index()
test_df

,index,price,country_Argentina,country_Australia,country_Canada,country_France,country_Italy,country_Spain,country_US,variety_Bordeaux-style Red Blend,...,vintage_2008,vintage_2009,vintage_2010,vintage_2011,vintage_2012,vintage_2013,vintage_2014,vintage_2015,vintage_2016,vintage_2017
0,27137,95,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,66081,20,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,47734,40,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,54891,60,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,67218,45,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19478,3244,60,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19479,39751,29,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
19480,35320,30,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19481,70090,50,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [33]:
X_train_scaled[0]

array([-2.11993901e-01, -2.01981036e-01, -1.28401544e-01, -5.20632789e-02,
       -4.67689061e-01,  2.71522365e+00, -2.12048370e-01, -1.22457900e+00,
       -2.65953241e-01, -1.26180720e-01, -3.45067791e-01, -1.24207005e-01,
       -3.88067623e-01, -1.77897982e-01, -1.84297229e-01, -1.76150355e-01,
       -1.30179850e-01, -4.18724599e-01, -3.26934002e-01, -1.36233269e-01,
       -1.83850139e-01, -1.94144189e-01, -1.77795599e-01, -2.07738904e-01,
        6.43545603e+00, -2.20798644e-01, -1.52770670e-01, -1.54810733e-01,
       -1.88279533e-01,  0.00000000e+00, -4.13636532e-03, -5.84975397e-03,
       -4.13636532e-03, -4.13636532e-03, -4.13636532e-03, -7.16451747e-03,
       -4.13636532e-03,  0.00000000e+00, -7.16451747e-03, -8.27294295e-03,
       -8.27294295e-03, -5.84975397e-03, -1.24099453e-02, -4.13636532e-03,
       -1.60219930e-02, -1.94047594e-02, -2.44781880e-02, -5.99054298e-02,
       -7.45452161e-02, -7.55824881e-02, -7.97083131e-02, -6.78704203e-02,
       -4.91755480e-02, -

In [4]:
loaded_model = tf.keras.models.load_model('Wine.h5')

c:\Users\juggl\anaconda3\envs\dev39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\juggl\anaconda3\envs\dev39\lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [5]:
# Compile the model
loaded_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = loaded_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

609/609 - 1s - 2ms/step - accuracy: 0.7652 - loss: 0.4717
Loss: 0.47168999910354614, Accuracy: 0.7651798725128174


In [35]:
loaded_model.save('Wine2.h5')

## Saving Predictions

In [36]:
import numpy as np

In [38]:
predictions = loaded_model.predict(X_test_scaled)
predictions

609/609 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([[0.81852865],
       [0.15674426],
       [0.2253179 ],
       ...,
       [0.31502482],
       [0.5366828 ],
       [0.86187303]], dtype=float32)

In [40]:
pred_df = pd.DataFrame(predictions, columns = ['prob'])

pred_df

,prob
0,0.818529
1,0.156744
2,0.225318
3,0.946267
4,0.766016
...,...
19478,0.847180
19479,0.258803
19480,0.315025
19481,0.536683


In [41]:
def threshold(prob): 
    if prob < .5: 
        return False
    else: 
        return True

In [42]:
pred_df['prediction'] = pred_df.apply(lambda x: threshold(x['prob']), axis = 1)
pred_df.head()

,prob,prediction
0,0.818529,True
1,0.156744,False
2,0.225318,False
3,0.946267,True
4,0.766016,True


In [44]:
pred_df['Actual'] = y_test
pred_df.head()

,prob,prediction,Actual
0,0.818529,True,False
1,0.156744,False,NaN
2,0.225318,False,False
3,0.946267,True,NaN
4,0.766016,True,NaN


In [45]:
y_test = y_test.reset_index(drop = True)
y_test

0        False
1         True
2        False
3         True
4        False
         ...  
19478    False
19479    False
19480    False
19481    False
19482     True
Name: target, Length: 19483, dtype: bool

In [47]:
pred_df['Actual'] = y_test
pred_df.head()

,prob,prediction,Actual
0,0.818529,True,False
1,0.156744,False,True
2,0.225318,False,False
3,0.946267,True,True
4,0.766016,True,False


In [48]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19483 entries, 0 to 19482
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prob        19483 non-null  float32
 1   prediction  19483 non-null  bool   
 2   Actual      19483 non-null  bool   
dtypes: bool(2), float32(1)
memory usage: 114.3 KB


In [68]:
scaled_data_df= pd.DataFrame(X_test_scaled)
scaled_data_df['index'] = test_df['index']
scaled_data_df

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,index
0,1.260076,-0.201981,-0.128402,-0.052063,-0.467689,-0.368294,-0.212048,0.816607,-0.265953,-0.126181,...,-0.289544,-0.326613,-0.324460,-0.386783,2.627696,-0.376307,-0.265695,-0.151121,-0.00585,27137
1,-0.438466,-0.201981,-0.128402,-0.052063,2.138173,-0.368294,-0.212048,-1.224579,3.760059,-0.126181,...,-0.289544,-0.326613,-0.324460,-0.386783,-0.380561,2.657403,-0.265695,-0.151121,-0.00585,66081
2,0.014478,-0.201981,-0.128402,-0.052063,2.138173,-0.368294,-0.212048,-1.224579,-0.265953,-0.126181,...,-0.289544,-0.326613,-0.324460,-0.386783,-0.380561,-0.376307,-0.265695,-0.151121,-0.00585,47734
3,0.467423,-0.201981,-0.128402,-0.052063,-0.467689,-0.368294,-0.212048,0.816607,-0.265953,-0.126181,...,-0.289544,-0.326613,-0.324460,-0.386783,-0.380561,-0.376307,3.763714,-0.151121,-0.00585,54891
4,0.127715,-0.201981,-0.128402,-0.052063,-0.467689,-0.368294,-0.212048,0.816607,-0.265953,-0.126181,...,-0.289544,-0.326613,-0.324460,-0.386783,2.627696,-0.376307,-0.265695,-0.151121,-0.00585,67218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19478,0.467423,-0.201981,-0.128402,-0.052063,-0.467689,2.715224,-0.212048,-1.224579,-0.265953,-0.126181,...,-0.289544,-0.326613,-0.324460,-0.386783,-0.380561,-0.376307,-0.265695,-0.151121,-0.00585,3244
19479,-0.234641,-0.201981,-0.128402,-0.052063,-0.467689,-0.368294,-0.212048,0.816607,-0.265953,-0.126181,...,-0.289544,-0.326613,3.082047,-0.386783,-0.380561,-0.376307,-0.265695,-0.151121,-0.00585,39751
19480,-0.211994,-0.201981,-0.128402,-0.052063,-0.467689,-0.368294,-0.212048,0.816607,-0.265953,-0.126181,...,-0.289544,-0.326613,-0.324460,-0.386783,-0.380561,-0.376307,-0.265695,-0.151121,-0.00585,35320
19481,0.240951,-0.201981,-0.128402,-0.052063,-0.467689,2.715224,-0.212048,-1.224579,-0.265953,-0.126181,...,-0.289544,3.061724,-0.324460,-0.386783,-0.380561,-0.376307,-0.265695,-0.151121,-0.00585,70090


In [65]:
scaled_data_df.to_csv('scaled_data_df.csv', index=False)

In [85]:
scaled_data_df.iloc[0,0:68]


0     1.260076
1    -0.201981
2    -0.128402
3    -0.052063
4    -0.467689
        ...   
63    2.627696
64   -0.376307
65   -0.265695
66   -0.151121
67   -0.005850
Name: 0, Length: 68, dtype: float64

In [91]:
test_1 = np.expand_dims(scaled_data_df.iloc[0,0:68], axis=0)
pred = loaded_model.predict(test_1)
pred[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


0.81852865

In [89]:
threshold(pred)

True